### Challenge 18: Implement CTR, the stream cipher mode

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)

<div class="alert alert-block alert-info">

The string:

```
L77na/nrFsKvynd6HzOoG7GHTLXsTVu9qvY/2syLXzhPweyyMTJULu/6/kXX0KSvoOLSFQ==
```
<br>
... decrypts to something approximating English in CTR mode, which is an AES block cipher mode that turns AES into a stream cipher, with the following parameters:
```
      key=YELLOW SUBMARINE
      nonce=0
      format=64 bit unsigned little endian nonce,
             64 bit little endian block count (byte count / 16)
```
CTR mode is very simple.

Instead of encrypting the plaintext, CTR mode encrypts a running counter, producing a 16 byte block of keystream, which is XOR'd against the plaintext.

For instance, for the first 16 bytes of a message with these parameters:
```
keystream = AES("YELLOW SUBMARINE",
                "\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00")
```
<br>
... for the next 16 bytes:
<br><br>
    
```
keystream = AES("YELLOW SUBMARINE",
                "\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00")
```
<br>
... and then:
<br><br>
    
```
keystream = AES("YELLOW SUBMARINE",
                "\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00")
```
<br>
CTR mode does not require padding; when you run out of plaintext, you just stop XOR'ing keystream and stop generating keystream.

Decryption is identical to encryption. Generate the same keystream, XOR, and recover the plaintext.

Decrypt the string at the top of this function, then use your CTR function to encrypt and decrypt other things.
This is the only block cipher mode that matters in good code.

Most modern cryptography relies on CTR mode to adapt block ciphers into stream ciphers, because most of what we want to encrypt is better described as a stream than as a sequence of blocks. Daniel Bernstein once quipped to Phil Rogaway that good cryptosystems don't need the "decrypt" transforms. Constructions like CTR are what he was talking about.
    
</div>

To implement AES CTR mode, let's just modify the existing AESEncrypt and AESDecrypt functions.  

For AESEncrypt, add the following code to handle CTR mode:
```python

    if mode != 'CTR':
        plaintext = PKCS7Pad(plaintext, 16)

    elif mode == 'CTR':

        if len(IV)==16 and (IV == [0]*16):
            # If all zeros nonce or none was passed in, set it to 8 x 0's
            IV = [0]*8

        elif len(IV) != 8:

           raise(ValueError('Nonce must be 8 bytes for CTR mode'))

        nonce = IV
        PT_Blocks = [plaintext[ii:ii+blockSize] for ii in
                     range(0, len(plaintext), blockSize)]

        for blk_idx, block in enumerate(PT_Blocks):

            AES_input = bytes(nonce) + int.to_bytes(blk_idx, 8, 'little')
            block_KEY = aes.encrypt(AES_input)
            ciphertext += bitwise_xor(block_KEY[0:len(block)], block)
```

For AESDecrypt, it's dead simple (Encrypt = Decrypt):
```
    elif mode == 'CTR':

        # Decrypt is the same as encrypt!
        plaintext = AESEncrypt(ciphertext, key, mode, IV)

    else:

        assert(f'Mode {mode} is not supported yet!')

    return(plaintext)
```

Let's try it out on the Challenge #18 ciphertext and parameters:

In [12]:
import cryptopals as cp
import base64 as b64

key = b'YELLOW SUBMARINE'
nonce = [0]*8

s_b64 = 'L77na/nrFsKvynd6HzOoG7GHTLXsTVu9qvY/2syLXzhPweyyMTJULu/6/kXX0KSvoOLSFQ=='
s = b64.b64decode(s_b64)

PT = cp.AESDecrypt(s, key, 'CTR', nonce)
print(PT.decode())

# Demonstrate that encrypt followed by decrypt works properly:
CT = cp.AESEncrypt(PT, key, 'CTR', nonce)
print(CT)
PT = cp.AESDecrypt(CT, key, 'CTR', nonce)
print(PT.decode())

# Could also just call encrypt to decrypt (they're the same!)
PT = cp.AESEncrypt(s, key, 'CTR', nonce)
print(PT.decode())


Yo, VIP Let's kick it Ice, Ice, baby Ice, Ice, baby 
b'/\xbe\xe7k\xf9\xeb\x16\xc2\xaf\xcawz\x1f3\xa8\x1b\xb1\x87L\xb5\xecM[\xbd\xaa\xf6?\xda\xcc\x8b_8O\xc1\xec\xb212T.\xef\xfa\xfeE\xd7\xd0\xa4\xaf\xa0\xe2\xd2\x15'
Yo, VIP Let's kick it Ice, Ice, baby Ice, Ice, baby 
Yo, VIP Let's kick it Ice, Ice, baby Ice, Ice, baby 


[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)